In [2]:
# data science
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import datetime

# API
import requests
import json

# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, text, inspect, func

In [3]:
# Create engine using the `covid.sqlite` database file
engine = create_engine("sqlite:///covid.sqlite")

In [4]:
# INSPECT to confirm existence

# Create the inspector and connect it to the engine
inspector_gadget = inspect(engine)

# Collect the names of tables within the database
tables = inspector_gadget.get_table_names()

# print metadata for each table
for table in tables:
    print(table)
    print("-----------")
    
    # get columns
    columns = inspector_gadget.get_columns(table)
    for column in columns:
        print(column["name"], column["type"])

    print()

covid
-----------
index BIGINT
Serial Number BIGINT
Country TEXT
Total Cases INTEGER
Total Deaths INTEGER
Total Recovered INTEGER
Active Cases INTEGER
Total Test INTEGER
Population INTEGER
Continent TEXT
country_code TEXT
latitude FLOAT
longitude FLOAT



In [5]:
# # user inputs
# user_region = 'All'
# user_min_attempts = 0

# # switch on user_region
# if user_region == 'All':
#     where_clause = "and 1=1"
# else:
#     where_clause = f"and region = '{user_region}'"

# build the query
query = f"""
    SELECT
        *
    FROM
        covid
        ;
"""

# execute query
df = pd.read_sql(text(query), con=engine)
df.head(10)

,index,Serial Number,Country,Total Cases,Total Deaths,Total Recovered,Active Cases,Total Test,Population,Continent,country_code,latitude,longitude
0,0,1,United States,104196861,1132935,101322779,1741147,1159832679,334805269,North America,US,37.090240,-95.712891
1,1,2,India,44682784,530740,44150289,1755,915265788,1406631776,Asia,IN,20.593684,78.962880
2,2,3,France,39524311,164233,39264546,95532,271490188,65584518,Europe,FR,46.227638,2.213749
3,3,4,Germany,37779833,165711,37398100,216022,122332384,83883596,Europe,DE,51.165691,10.451526
4,4,5,Brazil,36824580,697074,35919372,208134,63776166,215353593,South America,BR,-14.235004,-51.925280
5,5,6,Japan,32588442,68399,21567425,10952618,92144639,125584838,Asia,JP,36.204824,138.252924
6,6,7,S. Korea,30197066,33486,29740877,422703,15804065,51329899,Asia,KR,35.907757,127.766922
7,7,8,Italy,25453789,186833,25014986,251970,265478247,60262770,Europe,IT,41.871940,12.567380
8,8,9,UK,24274361,204171,24020088,50102,522526476,68497907,Europe,GB,55.378051,-3.435973
9,9,10,Russia,21958696,395108,21356008,207580,273400000,145805947,Asia,RU,61.524010,105.318756


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 197 entries, 0 to 196
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   index            197 non-null    int64  
 1   Serial Number    197 non-null    int64  
 2   Country          197 non-null    object 
 3   Total Cases      197 non-null    int64  
 4   Total Deaths     197 non-null    int64  
 5   Total Recovered  197 non-null    int64  
 6   Active Cases     197 non-null    int64  
 7   Total Test       197 non-null    int64  
 8   Population       197 non-null    int64  
 9   Continent        197 non-null    object 
 10  country_code     185 non-null    object 
 11  latitude         195 non-null    float64
 12  longitude        196 non-null    float64
dtypes: float64(2), int64(8), object(3)
memory usage: 20.1+ KB


In [7]:
df.describe()

,index,Serial Number,Total Cases,Total Deaths,Total Recovered,Active Cases,Total Test,Population,latitude,longitude
count,197.000000,197.000000,1.970000e+02,1.970000e+02,1.970000e+02,1.970000e+02,1.970000e+02,1.970000e+02,195.000000,196.000000
mean,105.619289,106.619289,3.384525e+06,3.398597e+04,3.251507e+06,9.903209e+04,3.505034e+07,3.934992e+07,19.594188,13.960951
std,64.350778,64.350778,1.019192e+07,1.120744e+05,9.845320e+06,7.947910e+05,1.220900e+08,1.471369e+08,23.781978,67.306978
min,0.000000,1.000000,1.403000e+03,1.000000e+00,4.380000e+02,0.000000e+00,7.850000e+03,4.965000e+03,-40.900557,-177.156097
25%,50.000000,51.000000,3.824000e+04,3.140000e+02,3.636600e+04,8.200000e+01,4.016220e+05,1.184817e+06,5.787949,-15.212844
50%,102.000000,103.000000,3.292270e+05,3.164000e+03,3.259340e+05,1.319000e+03,2.697814e+06,7.040745e+06,18.109581,18.687854
75%,160.000000,161.000000,1.746997e+06,1.692600e+04,1.731007e+06,1.174000e+04,1.580406e+07,2.917808e+07,40.106102,46.366989
max,224.000000,225.000000,1.041969e+08,1.132935e+06,1.013228e+08,1.095262e+07,1.159833e+09,1.412000e+09,71.706936,179.414413


In [8]:
query = """
            SELECT 
                "Total Cases",
                "Active Cases",
                "Total Recovered",
                Continent
                        
            From 
                covid
            Group by 
                 Continent
            Order by 
                "Total Cases" DESC;
            
            """
df3 = pd.read_sql(text(query), con=engine)
df3.head()
# df3.info()

,Total Cases,Active Cases,Total Recovered,Continent
0,104196861,1741147,101322779,North America
1,44682784,1755,44150289,Asia
2,39524311,95532,39264546,Europe
3,36824580,208134,35919372,South America
4,11295446,41060,11235771,Oceania


In [9]:
query = """
            SELECT 
                "Total Cases",
                "Active Cases",
                "Total Recovered",
               Country,
               Continent
                        
            From 
                covid
            Order by 
                "Total Cases" DESC;
            
            """
df3 = pd.read_sql(text(query), con=engine)
df3.head(10)

,Total Cases,Active Cases,Total Recovered,Country,Continent
0,104196861,1741147,101322779,United States,North America
1,44682784,1755,44150289,India,Asia
2,39524311,95532,39264546,France,Europe
3,37779833,216022,37398100,Germany,Europe
4,36824580,208134,35919372,Brazil,South America
5,32588442,10952618,21567425,Japan,Asia
6,30197066,422703,29740877,S. Korea,Asia
7,25453789,251970,25014986,Italy,Europe
8,24274361,50102,24020088,UK,Europe
9,21958696,207580,21356008,Russia,Asia


In [11]:
query = """
            SELECT 
                "Total Cases",
                "Active Cases",
                "Total Recovered",
               Country,
               Continent,
               Population
                        
            From 
                covid
            Order by 
                "Population" DESC;
            
            """
df3 = pd.read_sql(text(query), con=engine)
df3.head(10)

,Total Cases,Active Cases,Total Recovered,Country,Continent,Population
0,503302,118977,379053,China,Asia,1412000000
1,44682784,1755,44150289,India,Asia,1406631776
2,104196861,1741147,101322779,United States,North America,334805269
3,6730289,4264,6565208,Indonesia,Asia,279134505
4,1576313,6984,1538689,Pakistan,Asia,229488994
5,266463,3458,259850,Nigeria,Africa,216746934
6,36824580,208134,35919372,Brazil,South America,215353593
7,2037556,15420,1992694,Bangladesh,Asia,167885689
8,21958696,207580,21356008,Russia,Asia,145805947
9,7368252,429421,6606633,Mexico,North America,131562772


In [ ]:
# user inputs
user_continent = 'All'
user_min_attempts = 0

# switch on user_region
if user_continent == 'All':
    where_clause = "and 1=1"
else:
    where_clause = f"and Continent = '{user_continent}'"

